In [1]:
# Patrick Rall's Bicycle Polynomial Class
# (c) IBM Quantum 2023-2024

import numpy as np

class Poly():
    def __init__(self,d1,d2,terms):
        self.d1 = d1
        self.d2 = d2
        self.terms = [self.m_cast(m) for m in terms]

    def __repr__(self):
        if len(self.terms) == 0: return "0"
        labels = "xy"
        return " + ".join(["".join([
            labels[i] + "**" + str(term) if term > 1 else labels[i]
            for i,term in enumerate(elem)
            if term != 0
            ] if sum(elem) != 0 else "1") for elem in self.terms])

    ##################### Algebraic functions
    ## Poly() objects can be added, multiplied and exponentiated in the usual way

    def __add__(self,other):
        other = self.cast(other)

        out = []
        for elem in self.terms:
            if elem not in other.terms: out.append(elem)
        for elem in other.terms:
            if elem not in self.terms: out.append(elem)

        return Poly(self.d1,self.d2,out)

    def __radd__(self,other):
        return self.__add__(other)

    def __mul__(self,other):
        other = self.cast(other)

        out = []
        for elem1 in self.terms:
            for elem2 in other.terms:
                tup = self.m_mul(elem1,elem2)
                if tup in out: out.remove(tup)
                else: out.append(tup)

        return self.cast(out)

    def __rmul__(self,other):
        return self.__mul__(other)

    def __pow__(self,other):
        other = int(other)
        if other == 0: return self.cast(1)
        if other < 0:
            assert len(self.terms) == 1
            return self.T ** abs(other)

        return self * self**(other-1)

    # Transpose. Computes the transpose of the matrix representation
    # of the polynomial. Equivalent to taking the inverse of each monomial.
    @property
    def T(self):
        return self.cast([self.m_pow(m,-1) for m in self.terms])

    # Computes the matrix representation of the polynomial.
    def mat(self):
        def recur_mat(shape,term):
            if len(shape) == 0: return 1
            l = shape[0]
            a = term[0]
            S = np.zeros((l,l),dtype=int)
            for i in range(l):
                S[i,(i+a)%l] = 1
            return np.kron(S, recur_mat(shape[1:],term[1:]))

        out = np.zeros((self.d1*self.d2,self.d1*self.d2),dtype=int)
        for term in self.terms:
            out += recur_mat((self.d1,self.d2),term)
        return out%2

    # Turn anything into a Poly() object, such as 0, 1, a tuple (a,b) or list of tuples or a matrix
    def cast(self, other):
        if isinstance(other,Poly): return other
        if isinstance(other,int) and other == 0: return Poly(self.d1,self.d2,[])
        if isinstance(other,int) and other == 1: return Poly(self.d1,self.d2,[(0,0)])
        if isinstance(other,tuple): return Poly(self.d1,self.d2,[self.m_cast(other)])
        if isinstance(other,list): return Poly(self.d1,self.d2,[self.m_cast(m) for m in other])
        if isinstance(other,np.ndarray):
            assert len(other.shape) in [1,2]
            if len(other.shape) == 2:
                if other.shape[0] == 1: other = other[0,:]  # row vector
                elif other.shape[1] == 1: other = other[:,0]  # column vector
                else:
                    # matrix representation. want first row
                    assert other.shape[1] == self.d1*self.d2
                    other = other[0,:]
            assert other.shape[0] == self.d1*self.d2
            return Poly(self.d1,self.d2,[
                alpha for i,alpha in enumerate(self.monomials()) if other[i]%2 == 1
            ])

        print("Cast failed:", other)
        assert False

    ############# Utility functions for working with tuples (a,b) representing x**a * y**b

    # given (a,b) make sure they are are in the right range
    def m_cast(self, m):
        if m == 1: return (0,0)
        if isinstance(m,Poly):
            assert len(m.terms) == 1
            m = m.terms[0]

        a,b = m
        while a < 0: a += self.d1
        while a >= self.d1: a -= self.d1
        while b < 0: b += self.d2
        while b >= self.d2: b -= self.d2
        return (a,b)

    def m_mul(self, m1, m2):
        return self.m_cast(a+b for a,b in zip(self.m_cast(m1),self.m_cast(m2)))

    def m_div(self, m1, m2):
        return self.m_cast(a-b for a,b in zip(self.m_cast(m1),self.m_cast(m2)))

    def m_pow(self, m, pow):
        return self.m_cast(a*pow for a in self.m_cast(m))


 # Utility function for defining codes. Makes specifying the dimensions for x and y a bit easier.
def mapxy(d1,d2,fn):
    return fn(Poly(d1,d2,[(1,0)]), Poly(d1,d2,[(0,1)]))

codes = {
    "[72,12,6]": mapxy(6, 6, lambda x, y: [
        x**3 + y + y**2, y**3 + x + x**2,
    ]),
    "[90,8,10]": mapxy(15, 3, lambda x, y: [
        x**9 + y + y**2, 1 + x**2 + x**7,
    ]),
    "[108,8,10]": mapxy(9, 6, lambda x, y: [
        x**3 + y + y**2, y**3 + x + x**2,
    ]),
    "[144,12,12]": mapxy(12, 6, lambda x, y: [
        x**3 + y + y**2, y**3 + x + x**2,
    ]),
    "[288,12,12]": mapxy(12, 12, lambda x, y: [
        x**3 + y**2 + y**7, y**3 + x + x**2,
    ]),
}

if __name__ == "__main__":
    A, B = codes["[144,12,12]"]

    Xchecks = np.concatenate((A.mat(), B.mat()) , axis=1)
    Zchecks = np.concatenate((B.T.mat(), A.T.mat()) , axis=1)

    # rows are checks, columns are qubits
Xchecks.shape

(72, 144)

In [2]:
# generating x and z error strings -  starting with all zeros string as code word.

import numpy as np
num_datapoints = 10
error_prob = 0.3
error_string = np.zeros((num_datapoints,288), dtype = 'int')
error_string_avg = 0
for i in range(num_datapoints):
  r = np.random.uniform(size=288)
  error_string[i,:] = (r > (1 - error_prob)).astype(int)
  error_string_avg = error_string_avg + (np.sum(error_string[i,:])/288)
error_prob_test = print(error_string_avg/num_datapoints)

0.30659722222222224


In [3]:
# calculating the X and Z syndromes and concatenating them.

E_X = error_string[1,0:144].reshape(1,-1)

Zchecks_T = np.transpose(Zchecks)
syndrome_X = np.dot(E_X,Zchecks_T)%2

E_Z = error_string[1,144:].reshape(1,-1)

Xchecks_T = np.transpose(Xchecks)
syndrome_Z = np.dot(E_Z,Xchecks_T)%2

syndrome = np.concatenate((syndrome_X,syndrome_Z),axis=1)

In [4]:
# looping the above code to generate syndrome matrix.

syndrome_matrix = np.zeros((num_datapoints,144),dtype='int')

for i in range(num_datapoints):
    E_X = error_string[i,0:144].reshape(1,-1)
    Zchecks_T = np.transpose(Zchecks)
    syndrome_X = np.dot(E_X,Zchecks_T)%2
    E_Z = error_string[i,144:].reshape(1,-1)
    Xchecks_T = np.transpose(Xchecks)
    syndrome_Z = np.dot(E_Z,Xchecks_T)%2
    syndrome_matrix[i,:] = np.concatenate((syndrome_X,syndrome_Z),axis=1)

In [5]:
# Storing the error-syndrome pair in a matrix form (which is further fed into the FFNN model.)

error_syndrome = np.concatenate((error_string,syndrome_matrix),axis = 1)
error_syndrome.shape

(10, 432)

In [6]:
# FFNN model :

import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

from keras.models import Sequential
from keras.layers import Dense, Conv2D, Flatten
from tensorflow.keras.optimizers import SGD

# from pymatching import Matching

In [7]:
# defining the FFNN training model function.

def training_FFNN_model(error_syndrome, d=12, split=0.2, epochs=100, batch_size=2):

     x = error_syndrome[:, 288 : 432 ].astype(float)
     y = error_syndrome[:, 0 : 288 ]

     x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = split, random_state = 42, shuffle = True)

    ##reshaping input data in square matrix form
     train_len = x_train.shape[0]
     test_len = x_test.shape[0]


    ##FFNN model
     model_ffnn = Sequential()

     model_ffnn.add(Dense(64, input_dim = 144, activation="relu"))
     model_ffnn.add(Dense(288, activation="sigmoid"))

     print(model_ffnn.summary())


    ##model compilation
     model_ffnn.compile(loss = "binary_crossentropy",
                        optimizer = SGD(learning_rate=0.01),
                        metrics = ["accuracy"])

    ##model training
     model_ffnn.fit(x_train, y_train,
                    validation_split = 0.2,
                    epochs = epochs,
                    batch_size = batch_size,
                    verbose = 0)


    ##predictions and accuracy
     y_train_pred = model_ffnn.predict(x_train).round()
     train_acc = accuracy_score(y_train, y_train_pred)

     y_test_pred = model_ffnn.predict(x_test).round()
     test_acc = accuracy_score(y_test, y_test_pred)


     print("-"*40)
     print(f"Training Accuracy => {train_acc}")
     print(f"    Test Accuracy => {test_acc}")
     print("-"*40)

     return (x_test, y_test_pred, y_test)

In [8]:
# modifying Xchecks to its corresponding indices format.

stabilizer_set_xd12_f = np.zeros((72,6)) # cause we know the locality of the stabilizers
for index in range(Xchecks.shape[0]):
  j = 0
  for index1 in range(Xchecks.shape[1]):
    if Xchecks[index,index1] == 1 and j <= 5:
      stabilizer_set_xd12_f[index,j] = index1
      j = j+1

stabilizer_set_xd12 = stabilizer_set_xd12_f.astype(int)
stabilizer_set_xd12

array([[  1,   2,  18,  75,  78,  84],
       [  2,   3,  19,  76,  79,  85],
       [  3,   4,  20,  77,  80,  86],
       [  4,   5,  21,  72,  81,  87],
       [  0,   5,  22,  73,  82,  88],
       [  0,   1,  23,  74,  83,  89],
       [  7,   8,  24,  81,  84,  90],
       [  8,   9,  25,  82,  85,  91],
       [  9,  10,  26,  83,  86,  92],
       [ 10,  11,  27,  78,  87,  93],
       [  6,  11,  28,  79,  88,  94],
       [  6,   7,  29,  80,  89,  95],
       [ 13,  14,  30,  87,  90,  96],
       [ 14,  15,  31,  88,  91,  97],
       [ 15,  16,  32,  89,  92,  98],
       [ 16,  17,  33,  84,  93,  99],
       [ 12,  17,  34,  85,  94, 100],
       [ 12,  13,  35,  86,  95, 101],
       [ 19,  20,  36,  93,  96, 102],
       [ 20,  21,  37,  94,  97, 103],
       [ 21,  22,  38,  95,  98, 104],
       [ 22,  23,  39,  90,  99, 105],
       [ 18,  23,  40,  91, 100, 106],
       [ 18,  19,  41,  92, 101, 107],
       [ 25,  26,  42,  99, 102, 108],
       [ 26,  27,  43, 10

In [9]:
# modifying Zchecks to its corresponding indices format.

stabilizer_set_zd12_f = np.zeros((72,6)) # cause we know the locality of the stabilizers
for index in range(Zchecks.shape[0]):
  j = 0
  for index1 in range(Zchecks.shape[1]):
    if Zchecks[index,index1] == 1 and j <= 5:
      stabilizer_set_zd12_f[index,j] = index1
      j = j+1

stabilizer_set_zd12 = stabilizer_set_zd12_f.astype(int)
stabilizer_set_zd12

array([[  3,  60,  66,  76,  77, 126],
       [  4,  61,  67,  72,  77, 127],
       [  5,  62,  68,  72,  73, 128],
       [  0,  63,  69,  73,  74, 129],
       [  1,  64,  70,  74,  75, 130],
       [  2,  65,  71,  75,  76, 131],
       [  0,   9,  66,  82,  83, 132],
       [  1,  10,  67,  78,  83, 133],
       [  2,  11,  68,  78,  79, 134],
       [  3,   6,  69,  79,  80, 135],
       [  4,   7,  70,  80,  81, 136],
       [  5,   8,  71,  81,  82, 137],
       [  0,   6,  15,  88,  89, 138],
       [  1,   7,  16,  84,  89, 139],
       [  2,   8,  17,  84,  85, 140],
       [  3,   9,  12,  85,  86, 141],
       [  4,  10,  13,  86,  87, 142],
       [  5,  11,  14,  87,  88, 143],
       [  6,  12,  21,  72,  94,  95],
       [  7,  13,  22,  73,  90,  95],
       [  8,  14,  23,  74,  90,  91],
       [  9,  15,  18,  75,  91,  92],
       [ 10,  16,  19,  76,  92,  93],
       [ 11,  17,  20,  77,  93,  94],
       [ 12,  18,  27,  78, 100, 101],
       [ 13,  19,  28,  7

In [ ]:
# changing d's to 12

def postprocessing_low_level(x_test_test, y_pred, y_test, d =12, stabilizer_set_xd12, stabilizer_set_zd12):

    total = 0
    correct = 0
    logicalerrorcount = 0
    logicalXcount = 0
    logicalZcount = 0
    logicalYcount = 0

    ##confirming data length
    if (x_test.shape[0]!=y_pred.shape[0]) or (x_test.shape[0]!=y_test.shape[0]):
        print("Data length not equal")

    data_len = x_test.shape[0]

    for index in range(data_len):


        total += 1

        predictedqubit = y_pred[index].flatten().astype(int)
        truequbit = y_test[index].flatten().astype(int)

        qubitmatching = np.bitwise_xor(truequbit, predictedqubit)
        qubitmatching = qubitmatching.reshape(144, 2)
        qubitmatching_X = qubitmatching[:,0]
        qubitmatching_Z = qubitmatching[:,1]

        if_prediction_X_correct = np.bitwise_xor(qubitmatching_X,1).all()
        if_prediction_Z_correct = np.bitwise_xor(qubitmatching_Z,1).all()

        if_stabilizer_X = False
        if_stabilizer_Z = False

        nonzero_index_X = []
        for index in range(len(qubitmatching_X)):
            if (qubitmatching_X[index] == 1):
                nonzero_index_X.append(index)

        if nonzero_index_X in stabilizer_set_xd12:
            if_stabilizer_X = True


        nonzero_index_Z = []
        for index in range(len(qubitmatching_Z)):
            if (qubitmatching_Z[index] == 1):
                nonzero_index_Z.append(index)

        if nonzero_index_Z in stabilizer_set_zd12:
            if_stabilizer_Z = True





        if ((if_prediction_X_correct and if_prediction_Z_correct) or (if_stabilizer_X and if_stabilizer_Z) or (if_prediction_X_correct and if_stabilizer_Z) or (if_stabilizer_X and if_prediction_Z_correct)):  # when ML model predicts the exact error or its a stabilizer
            correct += 1

        else: # when prediction not correct, need to check if logical error

            qubitmatching_matrix_X = qubitmatching_X.reshape(12,12)
            qubitmatching_matrix_Z = qubitmatching_Z.reshape(12,12)
            qubitmatching_matrix_Z = np.transpose(qubitmatching_matrix_Z)

# need to find out the structure of logical operators for gross code.

            logicalz = [[1,1,1, 0,0,0, 0,0,0],
                        [0,0,0, 1,1,1, 0,0,0],
                        [0,0,0, 0,0,0, 1,1,1],
                        [0,0,1, 0,1,0, 1,0,0]]
            logicalz = np.array(logicalz)

            logicalx = [[1,0,0, 1,0,0, 1,0,0],
                        [0,1,0, 0,1,0, 0,1,0],
                        [0,0,1, 0,0,1, 0,0,1],
                        [1,0,0, 0,1,0, 0,0,1]]
            logicalx = np.array(logicalx)


            logicalX = False

            if (np.count_nonzero(qubitmatching_matrix_X[0] == 1) > 0
                and np.count_nonzero(qubitmatching_matrix_X[1] == 1) > 0
                and np.count_nonzero(qubitmatching_matrix_X[2] == 1) > 0):

                for i in range(len(logicalz)):
                    commutecheck = np.bitwise_and(qubitmatching_X, logicalz[i])
                    commutecount = 0
                    for c_i  in range (len(commutecheck)):
                        if (commutecheck[c_i]==1):
                            commutecount = commutecount+1

                    if ((commutecount%2)!=0):
                        logicalX=True
                        logicalXcount += 1
                        logicalerrorcount += 1
                        break

            ####################################

            logicalZ = False

            if (np.count_nonzero(qubitmatching_matrix_Z[0] == 1) > 0
                and np.count_nonzero(qubitmatching_matrix_Z[1] == 1) > 0
                and np.count_nonzero(qubitmatching_matrix_Z[2] == 1) > 0):

                for i in range(len(logicalx)):
                    commutecheck = np.bitwise_and(qubitmatching_Z, logicalx[i])
                    commutecount = 0
                    for c_i  in range (len(commutecheck)):
                        if (commutecheck[c_i]==1):
                            commutecount = commutecount+1

                    if ((commutecount%2)!=0):
                        logicalZ=True
                        logicalZcount += 1
                        logicalerrorcount += 1
                        break


            ##LogicalY check
            if (logicalX and logicalZ):
                logicalY = True
                logicalYcount += 1

                logicalX = False
                logicalZ = False
                logicalXcount -= 1
                logicalZcount -= 1

                logicalerrorcount -= 1

    print(f"       Total cases => {total}")
    print(f"    Correct by LLD => {correct}")
    print(f"Not correct by LLD => {total-correct}")
    print(f"ML model accuracy  => {(correct/total)*100} %")
    print("-"*40)

    print(f"    Logical X count => {logicalXcount}")
    print(f"    Logical Z count => {logicalZcount}")
    print(f"    Logical Y count => {logicalYcount}")
    print(f"Logical Error count => {logicalerrorcount}")
    print(f"Logical Error rate  => {logicalerrorcount / total}")
    print("-"*40)

    return (logicalerrorcount / total,
            logicalXcount / total,
            logicalZcount / total,
            logicalYcount / total,
            (correct/total)*100)

In [52]:
# calling the above defined functions.

x_test, y_pred, y_test = training_FFNN_model(error_syndrome, d=12, split = 0.2, epochs =100, batch_size = 2)

logicalerror, logicalX, logicalZ, logicalY, acc_lld = postprocessing_low_level(x_test, y_pred, y_test, d = 12, stabilizer_set_xd12, stabilizer_set_Zd12)

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_2 (Dense)             (None, 64)                9280      
                                                                 
 dense_3 (Dense)             (None, 288)               18720     
                                                                 
Total params: 28000 (109.38 KB)
Trainable params: 28000 (109.38 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None
1/1 [==============================] - 0s 23ms/step
----------------------------------------
Training Accuracy => 0.0
    Test Accuracy => 0.0
----------------------------------------
